In [23]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
 .master("spark://192.168.2.80:7077") \
 .appName("Viktor")\
 .config("spark.dynamicAllocation.enabled", True)\
 .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
 .config("spark.shuffle.service.enabled", False)\
 .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
 .config("spark.cores.max", 2)\
 .getOrCreate()

spark_context = spark_session.sparkContext


In [26]:
#Import CSV file containing all songs and features.
df=spark_session.read.load("hdfs://192.168.2.80:9000/data/SongCSV.csv",format="csv", sep=",", inferSchema="true", header="true")


In [27]:
df.printSchema()

root
 |-- SongNumber: integer (nullable = true)
 |-- SongID: string (nullable = true)
 |-- AlbumID: integer (nullable = true)
 |-- AlbumName: string (nullable = true)
 |-- ArtistID: string (nullable = true)
 |-- ArtistLatitude: double (nullable = true)
 |-- ArtistLocation: string (nullable = true)
 |-- ArtistLongitude: double (nullable = true)
 |-- ArtistName: string (nullable = true)
 |-- Danceability: double (nullable = true)
 |-- Duration: double (nullable = true)
 |-- KeySignature: integer (nullable = true)
 |-- KeySignatureConfidence: double (nullable = true)
 |-- Tempo: double (nullable = true)
 |-- TimeSignature: integer (nullable = true)
 |-- TimeSignatureConfidence: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Year: integer (nullable = true)



In [40]:
#Find out the amount of songs longer then 300 seconds.
df_1 = df.withColumn('Duration', df['Duration'].cast("float"))
df_1 = df_1.filter(df['Duration'] >= 300)
print("Amount of songs longer than 300 sec: "+str(df_1.count()))

Amount of songs longer than 300 sec: 1794


In [39]:
df.groupBy("ArtistName").count().orderBy('count', ascending=False).show()

+--------------------+-----+
|          ArtistName|count|
+--------------------+-----+
|    Mario Rosenstock|   13|
|        Phil Collins|   12|
|The Jackson South...|   12|
|        Sugar Minott|   12|
|           Aerosmith|   12|
|         Mikey Dread|   11|
|Nick Cave & The B...|   11|
|         Snow Patrol|   11|
|Bill & Gloria Gai...|   11|
|         Line Renaud|   11|
|           Radiohead|   11|
|Red Hot Chili Pep...|   10|
|                 Ayo|   10|
|         White Heart|   10|
|              Selena|   10|
|       Jacques Douai|   10|
|     Pumpkin Buzzard|   10|
|Stevie Ray Vaugha...|   10|
|             Shakira|   10|
|The Del McCoury Band|   10|
+--------------------+-----+
only showing top 20 rows



In [52]:
from pyspark.sql.functions import avg
grouped_df=df.groupBy("ArtistName")
avg_duration_df = grouped_df.agg(avg("Duration").alias("avg_duration"))
avg_duration_df.orderBy("avg_duration", ascending=False).show()

+--------------------+-----------------+
|          ArtistName|     avg_duration|
+--------------------+-----------------+
|   Ministry of Sound|        1686.7522|
|        Morton Gould|       1610.00444|
|     Kartik Seshadri|       1519.28118|
|   Frederic Galliano|       1394.75546|
|The Revolutionary...|       1253.92934|
|        Small Facers|       1141.08036|
|Naseebo Lal_ Arif...|       1037.68771|
|Arthur Fiedler;Ea...|        976.56118|
|Mickey And The St...|        914.02404|
|G\xc3\xb6teborgs ...|        904.80281|
|      Henri Pousseur|884.5735860000001|
|Saarbr\xc3\xbccke...|        827.66322|
|     Kurt Tepperwein|        819.87873|
|       Guitar Shorty|        817.37098|
|          Groundhogs|       812.590565|
|Wyclef Jean featu...|        809.58649|
|The Dreamtree Pro...|         798.1971|
|       Faiz Ali Faiz|785.9021333333334|
|Herbert von Karaj...|        770.35057|
| David Mingyue Liang|        757.99465|
+--------------------+-----------------+
only showing top

In [22]:
spark_session.stop()